***
## Learning without Forgetting - LwF

Experiment reproducing **Learning without Forgetting** method:  
hybrid of Distillation Networks and fine-tuning, which refers to the re-training with a low learning rate an already trained model M with new and more specific dataset, D<sub>new</sub>, with respect to the dataset, D<sub>old</sub>, with which the given model M was originally trained.

LwF, as opposed to other continual learning techniques, only uses the new data, so it assumes that past data used to pre-train the network is unavailable.  
It is a *transfer learning technique*.

`References:`
- Learning without Forgetting: https://arxiv.org/abs/1606.09282
- Three scenarios for continual learning: https://arxiv.org/abs/1904.07734
***

### Arguments to customize training
import argparse
parser = argparse.ArgumentParser()

parser.add_argument(**'--lwf_alpha'**, nargs='+', type=float,
                    default=[0, 0.5, 1.333, 2.25, 3.2],
                    help='Penalty hyperparameter for LwF. It can be either
                    a list with multiple elements (one alpha per 
                    experience) or a list of one element (same alpha 
                    for all experiences).')

parser.add_argument(**'--softmax_temperature'**, type=float, default=1,
                    help='Temperature for softmax used in distillation')

parser.add_argument(**'--lr'**, type=float, default=1e-3, help='Learning rate.')

parser.add_argument(**'--hs'**, type=int, default=256, help='MLP hidden size.')

parser.add_argument(**'--epochs'**, type=int, default=10,
                    help='Number of training epochs.')

parser.add_argument(**'--minibatch_size'**, type=int, default=128,
                    help='Minibatch size.')

parser.add_argument(**'--cuda'**, type=int, default=0,
                    help='Specify GPU id to use. Use CPU if -1.')

args = parser.parse_args()
***

In [1]:
# Testing framework and test runner
import unittest

import torch
from torch.nn import CrossEntropyLoss
from torch.optim import Adam, SGD

# Avalanche library from ContinualAI
import avalanche

from avalanche.benchmarks.datasets import MNIST
from avalanche.benchmarks.generators import nc_benchmark
from torchvision.transforms import Compose, ToTensor, Normalize, RandomCrop

# Loggers
from avalanche.logging import InteractiveLogger, TensorboardLogger

# Evaluation
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import accuracy_metrics, timing_metrics, forgetting_metrics

# Extras: Model and utils
from models import MLP # NN model (Multilayer perceptron)
from utils import arguments, get_average_metric, get_target_result

## LwF technique
class LwF(unittest.TestCase): #TestCase class

    #### Split MNIST benchmark
    def test_smnist(self, override_args=None):
        
        train_transform = Compose([
            RandomCrop(28, padding=4),
            ToTensor(),
            Normalize((0.1307,), (0.3081,)) ])

        test_transform = Compose([
            ToTensor(),
            Normalize((0.1307,), (0.3081,)) ])

        mnist_train = MNIST('./data/mnist', train=True, download=True, transform=train_transform)

        mnist_test = MNIST('./data/mnist', train=False, download=True, transform=test_transform)
        
        
        # -> BENCHMARK based on MNIST and Class-Incremental learning: Split-MNIST benchmark
        scenario = nc_benchmark(train_dataset=mnist_train,
                                test_dataset =mnist_test,
                                n_experiences=5,
                                task_labels = False)
        
               
        # --- Strategy instantiation --- # 
        # 1. Model
        # 2. Optimizer
        # 3. Loss function
        
        # -> ADDITIONAL ARGUMENTS allow to customize training
        args = arguments({ 'cuda': 0,              # GPU
                           'lwf_alpha': 1,         # Penalty hyperparameter for LwF
                           'lwf_temperature': 1,   # Temperature for softmax used in distillation
                           'epochs': 10,           # Training epochs
                           'layers': 1,             
                           'hidden_size': 256,     # MLP hidden size
                           'learning_rate': 0.001, # Learning rate
                           'train_mb_size': 128}, override_args) # Minibatch size

        # Set up and run CUDA operations,
        # if CUDA is available, utilize GPUs for computation.
        device = torch.device(f"cuda:{args.cuda}"
                              if torch.cuda.is_available() and args.cuda >= 0 
                              else "cpu")
        
        model = MLP(hidden_size=args.hidden_size, hidden_layers=args.layers)
        optimizer = SGD(model.parameters(), lr=args.learning_rate)
        criterion = CrossEntropyLoss()
        
        # ------------------------ LOG ------------------------ #
        # logging results over-time to examine the experiment in real-time
        loggers = []
        
        # log to Tensorboard
        loggers.append(TensorboardLogger())
        
        # Avalanche logging module, displays a progress bar during training and evaluation
        interactive_logger = avalanche.logging.InteractiveLogger()
        
        # -------------------- EVALUATION -------------------- #
        # Metrics of main interest to be tracked
        eval_plugin = EvaluationPlugin(
            accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
            #timing_metrics(epoch=True),
            #forgetting_metrics(experience=True, stream=True),
            #loggers=[InteractiveLogger()],
            loggers=loggers,
            benchmark=scenario
        )

        # -> CONTINUAL LEARNING STRATEGY: LwF
        cl_strategy = avalanche.training.LwF(model, optimizer, criterion,
                                             # additional arguments
                                             alpha=args.lwf_alpha, 
                                             temperature=args.lwf_temperature,
                                             train_mb_size=args.train_mb_size, 
                                             train_epochs=args.epochs,
                                             device=device,
                                             # evaluation
                                             evaluator=eval_plugin,
        )
        
        # --- Training loop --- #
        print('Starting experiment...')
        
        for experience in scenario.train_stream:
            print('Current experience {}, contains: {} patterns'.format(experience.current_experience, len(experience.dataset)))
            print('Current classes: ',experience.classes_in_this_experience)
            
            # Train
            cl_strategy.train(experience)
            print('Training completed')
            
            # Accuracy over the whole test set (no access to task-ID at inference time)
            print('Computing accuracy over the whole test set')
            res = cl_strategy.eval(scenario.test_stream)
            
        
        avg_stream_acc = get_average_metric(res)
        print(f"LwF-SplitTinyImageNet Average Stream Accuracy: {avg_stream_acc:.2f}")

### Run and Evaluate the experiment
- Create an instance of the strategy object
- Execute the strategy on a benchmark

In [2]:
# Create the strategy
s = LwF()

# Run the experiment with custom parameters
s.test_smnist()


Starting experiment...
Current experience 0, contains: 11686 patterns
Current classes:  [5, 7]
Training completed
Computing accuracy over the whole test set
Current experience 1, contains: 11907 patterns
Current classes:  [9, 2]
Training completed
Computing accuracy over the whole test set
Current experience 2, contains: 11765 patterns
Current classes:  [0, 4]
Training completed
Computing accuracy over the whole test set
Current experience 3, contains: 12049 patterns
Current classes:  [3, 6]
Training completed
Computing accuracy over the whole test set
Current experience 4, contains: 12593 patterns
Current classes:  [8, 1]
Training completed
Computing accuracy over the whole test set
LwF-SplitTinyImageNet Average Stream Accuracy: 0.28
